## Python で始める機器分析データの解析とケモメトリックス  
### 8.7. ハイパーパラメータのチューニングとクロスバリデーション

In [ ]:
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
data = pandas.read_csv("data3.csv", header=0, index_col=0).T
data.index = pandas.read_csv("prop3.csv", header=0, index_col=0).iloc[:, 2].values
data = ((data.T - data.T.mean()) / data.T.std()).T
from scipy.signal import savgol_filter
buff = data
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
data = pandas.DataFrame(buff, index=data.index, columns=data.columns)
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.6, random_state=8)

sklearn.pipeline.make_pipeline  
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html

In [ ]:
#p.148
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
PCR = make_pipeline(PCA(), LinearRegression())
PCR

In [ ]:
#p.148
PCR.set_params(pca__n_components=10)
model = PCR.fit(test.values, test.index)

sklearn.model_selection.GridSearchCV  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
#p.150, p.151
p1 = numpy.arange(1, 31, 1)
parm = {"pca__n_components": p1}
from sklearn.model_selection import GridSearchCV
search = GridSearchCV(PCR, parm, cv=3).fit(data.values, data.index)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
print(search.best_estimator_)
print(search.best_score_)

In [ ]:
#p.151
model = search.best_estimator_.fit(train.values, train.index)
calibration = model.predict(train.values)
validation = model.predict(test.values)

In [ ]:
pyplot.figure(figsize=(4, 4))
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()], c="black")
pyplot.scatter(train.index, calibration, label="calibration")
pyplot.scatter(test.index, validation, label="validation")
pyplot.legend()
pyplot.show()
from sklearn.metrics import mean_squared_error, r2_score
print("RMSEC =", numpy.sqrt(mean_squared_error(train.index, calibration)))
print("R^2 =", r2_score(train.index, calibration))
print("")
print("RMSEV =", numpy.sqrt(mean_squared_error(test.index, validation)))
print("R^2 =", r2_score(test.index, validation))

(c) Shigeaki Morita